In [1]:
import statsmodels.stats.power as smp
from scipy.stats import ttest_1samp
import numpy as np

In [2]:
alpha = 0.01
p_aware = 0.05

In [3]:
power_analysis = smp.TTestPower()
sample_size = power_analysis.solve_power(
    effect_size = 0.451, 
    power = 0.95, 
    alpha = alpha, 
    alternative = 'larger'
)
sample_size = np.round(sample_size).astype(int)
print('Sample size:', sample_size)

Sample size: 80


/home/johnv/anaconda3/envs/niseq/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6832: RuntimeWarning: divide by zero encountered in _nct_sf
  return np.clip(_boost._nct_sf(x, df, nc), 0, 1)


In [4]:
def simulate_subject(p_aware):
    
    seed = np.random.randint(0, 100000000)
    rng = np.random.default_rng(seed)
    
    n_trials = 40 # per condition, standard for intentional binding
    sd = 92 # standard deviation of trial-by-trial estimates around sub mean
    effect_size = 28 + rng.normal(0, 76) # jittered by population variance
    
    est_baseline = rng.normal(0, sd, n_trials)
    est_unaware = rng.normal(0, sd, n_trials)
    est_aware = rng.normal(effect_size, sd, n_trials)
    aware = rng.uniform(size = n_trials) < p_aware
    est_operant = np.choose(aware, [est_unaware, est_aware])
    return est_operant.mean() - est_baseline.mean()

def simulate(p_aware, n_subjects, alpha):
    deltas = [simulate_subject(p_aware) for i in range(n_subjects)]
    res = ttest_1samp(deltas, popmean = 0, alternative = 'greater')
    return res.pvalue < alpha
    

In [5]:
# find combined false positive rate by simulation
np.random.seed(0)
n_simulations = 5000
ct = 0
for i in range(n_simulations):
    ct += simulate(p_aware, sample_size, alpha)
print(ct / n_simulations)

0.0426


In [6]:
accuracy = lambda p_aware: .5*(1 - p_aware) + 1.*p_aware
accuracy(p_aware) # accuracy corresponding to p_aware

0.525

In [10]:
np.round(60. / 10.).astype(int)

6

In [30]:
np.random.randint(0, 5)

4

In [31]:
600/560

1.0714285714285714